In [1]:
from __future__ import absolute_import, division, print_function

import os
import pickle
from six.moves import urllib

import tflearn
from tflearn.data_utils import *

path = "all_siir.txt"
char_idx_file = 'char_idx.pickle'

maxlen = 30

char_idx = None
if os.path.isfile(char_idx_file):
    print('Loading previous char_idx')
    char_idx = pickle.load(open(char_idx_file, 'rb'))

X, Y, char_idx = \
    textfile_to_semi_redundant_sequences(path, seq_maxlen=maxlen, redun_step=3,
                                         pre_defined_char_idx=char_idx)

pickle.dump(char_idx, open(char_idx_file,'wb'))

g = tflearn.input_data([None, maxlen, len(char_idx)])
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512)
g = tflearn.dropout(g, 0.5)
g = tflearn.fully_connected(g, len(char_idx), activation='softmax')
g = tflearn.regression(g, optimizer='adam', loss='categorical_crossentropy',
                       learning_rate=0.001)

m = tflearn.SequenceGenerator(g, dictionary=char_idx,
                              seq_maxlen=maxlen,
                              clip_gradients=5.0)



Loading previous char_idx
Vectorizing text...
Text total length: 18,397,191
Distinct chars   : 69
Total sequences  : 6,132,387


In [2]:
m.load("last.tflearn")

INFO:tensorflow:Restoring parameters from /output/last.tflearn


In [3]:
test = ""

In [ ]:
for i in range(100):
    m.fit(X, Y, validation_set=0.001, batch_size=256,
          n_epoch=1, run_id='siir')
    
    m.save(str(i)+".tflearn")
    
    seed = random_sequence_from_textfile(path, maxlen)
    test += "Epoch:" + str(i) + "\n"
    test += (m.generate(200, temperature=0.5, seq_seed=seed))
    
    with open("test.txt", 'w') as test_text:
        test_text.write(test)

Training Step: 559171  | total loss: 1.81675 | time: 2435.236s
| Adam | epoch: 016 | loss: 1.81675 -- iter: 2963200/6126254


In [3]:
m.save("last.tflearn")

INFO:tensorflow:/output/last.tflearn is not in all_model_checkpoint_paths. Manually adding it.
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
